In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
sylt_ids = np.load('../../contrastive_learning/sylt_ids.npz')['ids']

In [3]:
def mag_to_flx(mag):
    """
    Input: an AB magnitude
    Output: a flux in microjansky (muJy)
    Works with np arrays and pd Series
    """
    return 10**(29-(48.60/2.5)) * 10**(-mag/2.5)

def read_lephare_spectrum_file(id):
    """
    Reads the file containing the output spectra for the inputted ID
    """
    with open(f'output_spectra/Id{str(id)[-9:]}.spec','r') as f:
        specfile_lines = f.read().split('\n')
    specfile_lines = [row.split() for row in specfile_lines]
    return specfile_lines

def get_lephare_spectrum(id, i): # i from 0 to 2
    """
    Inputs: An ID, and an integer in [0,1,2]
    Outputs: wavelengths, and a spectrum in muJy. The spectrum is the 
        LePHARE best fit spectrum for:
            0: Galaxy
            1: Quasar
            2: Star
    """
    file_lines = read_lephare_spectrum_file(id)
    spectra = np.array(file_lines[171:][:-1]).astype('float32')
    restart_indices = np.where(np.diff(spectra, axis=0)[:,0]<0)[0]
    spec_list = np.split(spectra, restart_indices+1, axis=0)
    x,y_mag = spec_list[i][:,0],spec_list[i][:,1]
    x,y_mag = x[(y_mag<100) & (y_mag!=0)], y_mag[(y_mag<100)&(y_mag!=0)] # Clipping ceilinged SED values
    y = mag_to_flx(y_mag)
    return x,y

Now the various wavelength and flux arrays generated by the above function are not of the same size (not just because some values have been clipped, it seems LePHARE always outputs non-standard lengths of spectra, so that's just great). In order to store the spectra in a nice way, and because we're only interested in a certain wavelength range anyway, we choose a standardised set of wavelength values between 0.4$\mu$m to 6$\mu$m and interpolate the LePHARE spectra.

In [5]:
num_samples = 10000
wavelength_samples = np.linspace(4e3, 6e4, num_samples)
spectrum_array = np.zeros((len(sylt_ids), 3, num_samples)) # ID x Spectrum Type (G/Q/S) x wavelength

for i, idd in tqdm(enumerate(sylt_ids), total=len(sylt_ids)):
    for obj_num in range(3):
        wavs, flxs = get_lephare_spectrum(idd,obj_num)
        flux_samples = np.interp(wavelength_samples,
                                 wavs, flxs)
        spectrum_array[i,obj_num,:] = flux_samples

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 30.60it/s]


In [6]:
with open('lephare_output.out','r') as f:
    lephare_output_text = f.read()
lephare_df = pd.DataFrame([line.split() for line in lephare_output_text.split('\n')[55:]],
                          columns = ['COADD_OBJECT_ID',
                                     'Z_BEST',
                                     'Z_BEST68_LOW',
                                     'Z_BEST68_HIGH',
                                     'Z_ML',
                                     'CHI_BEST',
                                     'MOD_BEST',
                                     'MAG_ABS_BEST',
                                     'PDZ_BEST',
                                     'SCALE_BEST',
                                     'DIST_MOD_BEST',
                                     'NBAND_USED',
                                     'Z_SEC',
                                     'CHI_SEC',
                                     'MOD_SEC',
                                     'AGE_SEC',
                                     'Z_QSO',
                                     'CHI_QSO',
                                     'MOD_QSO',
                                     'MAG_ABS_QSO',
                                     'DIST_MOD_QSO',
                                     'MOD_STAR',
                                     'CHI_STAR',
                                     'CONTEXT',
                                     'ZSPEC'
                                    ]
                         )[:-1].set_index('COADD_OBJECT_ID')
lephare_df = pd.DataFrame(
    lephare_df.loc[[str(int(str(idd)[-9:])) for idd in sylt_ids]].to_numpy(),
    columns = lephare_df.columns,
    index=sylt_ids)
lephare_df.to_csv('lephare_output_data.csv')
lephare_df

,Z_BEST,Z_BEST68_LOW,Z_BEST68_HIGH,Z_ML,CHI_BEST,MOD_BEST,MAG_ABS_BEST,PDZ_BEST,SCALE_BEST,DIST_MOD_BEST,...,AGE_SEC,Z_QSO,CHI_QSO,MOD_QSO,MAG_ABS_QSO,DIST_MOD_QSO,MOD_STAR,CHI_STAR,CONTEXT,ZSPEC
980453716,1.0327,1.0380,1.0409,1.0400,0.137578E+04,1,-24.318,100.000,0.620859E+06,0.441881E+02,...,-.990000E+02,6.1000,0.109059E+03,17,-28.201,0.488499E+02,207,0.636932E+03,508.0000,-.990000E+02
1143273115,0.9994,0.9994,1.0006,-99.0000,0.199684E+04,1,-24.272,100.000,0.604926E+06,0.441000E+02,...,-.990000E+02,6.1000,0.377818E+03,5,-27.693,0.488499E+02,209,0.117940E+04,511.0000,-.990000E+02
1197311621,0.9959,0.9955,1.0029,0.9996,0.521169E+03,1,-23.425,100.000,0.274958E+06,0.440906E+02,...,-.990000E+02,6.2000,0.238606E+02,16,-27.341,0.488909E+02,209,0.419063E+03,508.0000,-.990000E+02
1228745162,0.9550,0.9596,0.9602,-99.0000,0.430904E+04,1,-24.410,100.000,0.679011E+06,0.439781E+02,...,-.990000E+02,6.0000,0.772263E+02,15,-27.585,0.488082E+02,211,0.125936E+04,510.0000,-.990000E+02
1271696125,1.1048,1.1160,1.1206,1.1197,0.102673E+04,1,-24.490,100.000,0.715978E+06,0.443695E+02,...,-.990000E+02,6.5000,0.141360E+03,16,-28.203,0.490099E+02,207,0.774716E+03,508.0000,-.990000E+02
1456016600,0.8690,0.8773,0.8808,-99.0000,0.248079E+04,1,-23.439,100.000,0.272784E+06,0.437253E+02,...,-.990000E+02,6.0000,0.348033E+02,20,-27.127,0.488082E+02,211,0.615816E+03,510.0000,-.990000E+02
1470729144,0.9535,0.9585,0.9608,-99.0000,0.173261E+04,1,-23.513,100.000,0.295955E+06,0.439738E+02,...,-.990000E+02,6.0000,0.762809E+01,22,-26.967,0.488082E+02,210,0.617169E+03,510.0000,-.990000E+02
1471313744,1.0144,0.9998,1.0013,-99.0000,0.582916E+04,1,-24.495,100.000,0.770518E+06,0.441402E+02,...,-.990000E+02,6.1000,0.485497E+03,22,-27.779,0.488499E+02,210,0.489300E+04,508.0000,-.990000E+02
1513805078,0.9632,0.9592,0.9610,-99.0000,0.209640E+04,1,-23.624,100.000,0.336211E+06,0.440011E+02,...,-.990000E+02,6.0000,0.353117E+02,11,-27.358,0.488082E+02,210,0.696056E+03,510.0000,-.990000E+02
1599741416,0.9704,0.9593,0.9623,0.9600,0.950755E+03,1,-23.593,100.000,0.333020E+06,0.440212E+02,...,-.990000E+02,5.9600,0.295227E+02,16,-27.243,0.487913E+02,185,0.303842E+03,510.0000,-.990000E+02


In [7]:
# A 6690x3 array (IDs x G/Q/S)
chi2s = lephare_df.loc[sylt_ids][['CHI_BEST','CHI_QSO','CHI_STAR']].to_numpy().astype('float32')

In [8]:
np.savez_compressed('lephare_spectra_chi2s.npz',
                    ids=sylt_ids,
                    wavelengths = wavelength_samples,
                    spectra=spectrum_array,
                    chi2s=chi2s)